In [1]:
# Importações
import os
import cv2
import pandas as pd
import math
import numpy as np
from keras.utils import np_utils
from keras.layers import Dense, InputLayer, Dropout
from skimage.transform import resize
from keras.applications.vgg16 import preprocess_input, VGG16
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from matplotlib import pyplot as plt
from dotenv import load_dotenv
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

# Carregar variáveis de ambiente
load_dotenv('amb_var.env')

False

In [2]:
# Separação de frames do vídeo de treinamento
count = 0
videoFile = r"C:\Users\pytho\Documents\GitHub\Icity\iVision\Model\Acidentes-ICity Train - Made with Clipchamp.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5)  # frame rate
while(cap.isOpened()):
    frameId = cap.get(1)  # current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename = "%d.jpg" % count
        count += 1
        cv2.imwrite(filename, frame)
cap.release()

In [3]:
# Leitura do CSV de classificação
data = pd.read_csv('mapping_final.csv')
X = [plt.imread('' + img_name) for img_name in data.Image_ID]
X = np.array(X)

y = data.Class
dummy_y = np_utils.to_categorical(y)

MemoryError: 

In [ ]:
# Preprocessamento das imagens
X = [resize(img, preserve_range=True, output_shape=(224, 224)).astype(int) for img in X]
X = np.array(X)
X = preprocess_input(X, data_format=None)

In [ ]:
# Divisão em treino e teste
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

In [ ]:
# Modelo
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)

X_train = X_train.reshape(X_train.shape[0], 7 * 7 * 512)
X_valid = X_valid.reshape(X_valid.shape[0], 7 * 7 * 512)
train = X_train / X_train.max()
X_valid = X_valid / X_train.max()

model = Sequential()
model.add(InputLayer((7 * 7 * 512,)))
model.add(Dense(units=1024, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit_generator(datagen.flow(train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, epochs=100,
                    validation_data=(X_valid, y_valid), callbacks=[early_stop])

model.save('Model2')

In [ ]:
# Teste
count = 0
videoFile = "Accident-1.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5)  # frame rate
while(cap.isOpened()):
    frameId = cap.get(1)  # current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename = "test%d.jpg" % count
        count += 1
        cv2.imwrite(filename, frame)
cap.release()

In [ ]:
# Leitura de CSV de teste e preprocessamento
test = pd.read_csv('test.csv')
test_image = [plt.imread('' + img_name) for img_name in test.Image_ID]
test_img = np.array(test_image)
test_image = [resize(img, preserve_range=True, output_shape=(224, 224)).astype(int) for img in test_img]
test_image = preprocess_input(np.array(test_image), data_format=None)
test_image = base_model.predict(test_image)
test_image = test_image.reshape(test_image.shape[0], 7 * 7 * 512)
test_image = test_image / test_image.max()

predictions = model.predict(test_image)

In [ ]:
# Teste em tempo real
cap = cv2.VideoCapture('Untitled video - Made with Clipchamp.mp4')
i = 0
flag = 0
while(True):
    ret, frame = cap.read()
    if ret == True:
        if i % 5 == 0:  # Predição a cada 5 frames
            if predictions[int(i / 15) % 9][0] < predictions[int(i / 15) % 9][1]:
                predict = "Sem Acidente"
            else:
                predict = "Acidente"
                flag = 1
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, predict, (50, 50), font, 1, (0, 255, 255), 3, cv2.LINE_4)
        cv2.imshow('Frame', frame)
        i += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
if flag == 1:
    print('Acidente')

cap.release()
cv2.destroyAllWindows()


In [6]:
# Limpeza de Imagens
def apagar_imagens():
    for i in range(5051):
        nome_arquivo = f"{i}.jpg"
        if os.path.exists(nome_arquivo):
            os.remove(nome_arquivo)
    for i in range(9):
        nome_arquivo

apagar_imagens()